# Load the Data

In [44]:
import numpy as np

In [45]:
# create a list of all the texts
texts = []
with open('testset.txt', 'r') as f:
    for line in f:
        elements = line.strip().split('\t')
        texts.append(elements[0].replace('\n', ''))
        texts.append(elements[1])

In [46]:
for text in texts:
    print(text)

Tasting it is the only reliable way.
The way you have it is fine.
I think it probably depends on your money.
It depends on your country.
You need to read a lot to know what you like and what you don't.
You don't have to know.
Obviously, the best book for you depends a lot on what you are looking for.
The answer will depend of course on what you're looking for in a vacation.
I've had this same problem.
I had the same problem as you.
If you are not sure how to do it, don't do it at all.
Also, if you die, you don't have to repay, don't know if it helps.
It is more deciding what you should do and what you should get others to do.
What can you do and what should you do to grow more?
You need to add this to your resume because it is the only position you have had.
You may want it, but the process given to you is what you have to work within.
I remained under the banyan tree, exhausted by my daily ritual of dragooning the men every two hours. 
I remained under the banyan tree, exhausted by my

# Create Shingles for each text

In [47]:
shingles_per_text = []
k = 4
for text in texts:
    shingles_per_text.append(set([text[i:i + k] for i in range(len(text) - k + 1)]))

print(shingles_per_text[0])

{' rel', 'ting', 'y re', ' onl', 'e wa', 's th', 'ing ', 'stin', 'asti', 'le w', ' way', 't is', 'able', 'is t', 'elia', 'only', 'reli', 'he o', 'Tast', 'ble ', 'liab', ' it ', 'iabl', 'ng i', 'way.', 'ly r', 'it i', 'g it', ' the', 'the ', 'nly ', ' is ', 'e on'}


# Create Bit-Vector for each text (Matrix of Bit-Vectors [f. 51])

In [48]:
# create a set of all shingles
all_shingles_of_file = set()
for shingle_of_text in shingles_per_text:
    all_shingles_of_file.update(shingle_of_text)
print(all_shingles_of_file)

{'ant ', 'xhau', 'babl', 'now.', 've t', ' sam', 'g fo', 'nlin', 'r re', 'prob', ' a l', 'e ho', 'ad t', ' you', 'mone', 'e ca', 'ok f', 'on y', 'king', 't it', 'ntry', ' it,', 'ther', 'how ', 'e an', 've h', 'may ', 'iven', ', ex', 'this', 'lps.', ', yo', 'ur r', 'so, ', 's gi', ' tre', ' ans', 've i', 'on o', 'stin', 'answ', 's ev', 'ore ', 'ecau', 'rs. ', 'd un', 'if y', 'work', 'an t', 'urs.', 'onli', ' a t', 'okin', 'elia', ' if ', 'k fo', 'more', 'w mo', 'know', 'a to', 'here', ' nee', 'ld g', "re's", ' do.', 't pr', 'o ho', 'd to', 'nk i', 'othe', ' eve', 'If y', 'ily ', 'ly, ', "u're", 'yan ', 'sly,', 'aria', 'ends', ' the', 't su', 'ion ', 'ned ', 'ld d', 'r wi', 'ad a', 'ou i', 'tree', 'atio', 'ng t', 'ut t', 'le w', 'ecid', 'sure', 'is s', 'f va', 'a va', 'need', 'ause', 't bo', ' by ', 's yo', 'at c', 'iabl', 're n', 'hat ', ' lik', 'our ', 'al o', 'usly', "e's ", 'thin', 'itua', 'ney.', 'n ev', "you'", 'ay y', 's th', 'asti', 'e no', 'rema', 'nd o', 'on w', "I've", ', do',

In [49]:
# create Bit-Vector for each text -> this will yield the matrix of the bit-vectors
bit_Matrix = []
for shingle_of_text in shingles_per_text:
    bit_vector_for_text = []
    
    for shingle_of_file in all_shingles_of_file:
        
        if shingle_of_file in shingle_of_text:
            bit_vector_for_text.append(1)
        else:
            bit_vector_for_text.append(0)
            
    bit_Matrix.append(bit_vector_for_text)

In [50]:
# nur um zz zeigen, dass es funktioniert
for vector in bit_Matrix:
    print(vector[0:16])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Calculate MinHash-Signature for each text

# Create Hash-Functions

In [59]:
# generate n hash functions (random permutations)
n = 100
hash_functions = []
for h in range(n):
    perm = np.random.permutation(len(all_shingles_of_file))
    hash_functions.append(perm)

print(hash_functions[0])

[621 175 574 249 439 363 519   7 264 391 256 569 287  76 370 366 413 253
  20 342 499 176   9 577 526 431  16 218 277 289  51  75   6 198  96 310
 319 440 183 243  94   8 550 357 367 296 261 173 300 451 371  17 461  97
 622 315 589 442 293 449 571   0 332 435 316 116 128 563 153 488 229 263
 590 216 373 361  30 103 164 497 533  32 374 309 324  45  67 123 358 548
 354 520 408  41 393 114 388 257 294 196 510 348 375 214 547 194 535 549
 436 611 447 562 469 450 302 207 384 567 239 311 297 250 338 507 344 396
   4  35  62 438 146 322 336  14 406 508 434  57 185 291 543 304 529 275
  65  98 140 131 190 129 134 335 270 493  83  73 599 428 516 504 356 328
 352  95 286 292 506 227 573  66 433 583 187 339 290 578 108 232 209 189
 612 323 139 109 423 166 441 455 205 552 399 530 527  39 340 295 409 446
  37 593 496 597 151  80 359 518  71  81 584  99 237 211 387 502  89 203
 337 407 598  13 607 517 514 238  54 372 610 169 111 466 420 538 561 259
 231 260 532 405 606  38  52  36 512 195 414 285 18

# Calculate MinHash-Signature for each text (MinHash Matrix)

In [62]:
signature_matrix = []
for hash_function in hash_functions:
    signature_vector = []
    for bit_vector in bit_Matrix:
        # find the first 1 in the bit vector
        # we don't permutate the original bit vector, we instead use the hash function and check the bit vector at the permutated index
        for i in range(len(bit_vector)):
            if bit_vector[hash_function[i]] == 1:
                signature_vector.append(hash_function[i])
                break
    signature_matrix.append(signature_vector)

In [61]:
for signature_vector in signature_matrix:
    print(signature_vector[0:16])

[310, 319, 9, 20, 431, 300, 76, 621, 439, 519, 391, 175, 391, 569, 439, 256]
[474, 284, 107, 107, 157, 151, 373, 157, 376, 291, 421, 58, 157, 157, 225, 552]
[91, 541, 294, 294, 192, 426, 412, 412, 519, 519, 421, 372, 192, 57, 239, 262]
[492, 13, 65, 187, 428, 426, 458, 422, 5, 79, 214, 136, 607, 611, 492, 142]
[349, 349, 107, 107, 106, 284, 588, 618, 408, 291, 288, 284, 349, 566, 506, 240]
[614, 319, 475, 460, 380, 380, 138, 231, 29, 344, 154, 299, 348, 348, 239, 105]
[590, 37, 475, 293, 606, 465, 447, 121, 439, 590, 308, 549, 370, 370, 590, 590]
[180, 396, 390, 390, 446, 4, 390, 233, 381, 381, 465, 561, 22, 128, 250, 586]
[522, 37, 347, 17, 317, 275, 317, 317, 347, 205, 414, 124, 317, 413, 347, 317]
[614, 605, 487, 430, 257, 465, 458, 96, 205, 205, 119, 193, 298, 128, 605, 86]
[290, 493, 501, 187, 432, 562, 618, 579, 123, 270, 421, 562, 425, 603, 25, 358]
[614, 396, 293, 293, 67, 523, 508, 356, 12, 291, 545, 617, 163, 413, 67, 47]
[174, 578, 278, 278, 151, 151, 125, 422, 218, 531, 119

In [ ]:
signature_similarity = []
